In [1]:
import pandas as pd
import numpy as np
import os
from io import StringIO 
import json
from collections import Counter


from enum import Enum

import sys  
sys.path.append( '/Users/roshansk/Documents/GitHub/langchain/libs/langchain')
sys.path.append( '/Users/roshansk/Documents/GitHub/langchain/libs/experimental/')
sys.path.append( '/Users/roshansk/Documents/GitHub/langchain/libs/partners/openai')
sys.path.append( '/Users/roshansk/Documents/GitHub/langchain/libs/community/langchain_community/')


from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import JsonOutputParser
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS, Chroma, Milvus
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.vectorstores.faiss import DistanceStrategy
from langchain_pinecone import PineconeVectorStore
import langchain_community
from langchain_core.vectorstores import VectorStore
from langchain_core.documents import Document


from typing import (
    Any,
    Callable,
    Dict,
    Iterable,
    List,
    Optional,
    Sized,
    Tuple,
    Union,
)

%load_ext autoreload
%autoreload 2

In [2]:
def get_document_by_id_chroma(store: VectorStore, ids : int | str | List[int|str]) -> List[Document]:

    if isinstance(ids, list):
        output = store._collection.get(ids = ids)
    else:
        output = store._collection.get(ids = [ids])

    num_results = len(output['ids'])

    output_docs = []
    
    if num_results>0:

        for i in range(num_results):
            metadata = output['metadatas'][i]
            page_content = output['documents'][i]

            output_docs.append(Document(page_content=page_content, metadata=metadata))

    return output_docs